In [1]:

import numpy as np
from datasets import Dataset, load_dataset
import datasets
import pandas as pd
from datasets import Dataset
import random
import numpy as np
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer
import torch
from transformers import BitsAndBytesConfig
from torch import nn
from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Tuple, Union
import torch.nn.functional as F


/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:654: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/opt/conda/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
dataset = load_dataset('csv', data_files='paradetox.tsv', delimiter='\t', split='train')

tokenizer = AutoTokenizer.from_pretrained('model_with_dpo_onlypass')

def preprocess_function(examples):
    text = examples['toxic']
    examples['input_ids'] = tokenizer("paraphrase: " + text, max_length=128, truncation=True)['input_ids']
    # example['labels'] = tokenizer(text_target=neutral1, max_length=max_target_length, truncation=True)['input_ids']
    # new_examples['labels'].append(labels['input_ids'])
    return examples

# dataset = dataset.map(preprocess_function, remove_columns=['id','toxic','severe_toxic','obscene','threat','insult','identity_hate'])

In [3]:
print(dataset)

dataset = dataset.map(preprocess_function, remove_columns=['neutral1', 'neutral2', 'neutral3']).select(range(500))
print(dataset)
# dataset = dataset.map(preprocess_function, remove_columns=['id','toxic','severe_toxic','obscene','threat','insult','identity_hate'])

Dataset({
    features: ['toxic', 'neutral1', 'neutral2', 'neutral3'],
    num_rows: 11927
})
Dataset({
    features: ['toxic', 'input_ids'],
    num_rows: 500
})


In [ ]:
# model_name = 'model_with_dpo_second'
model_name = 'model_with_dpo_onlypass'
# model_name = 'humarin/chatgpt_paraphraser_on_T5_base'

# my_model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')
my_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


input_texts = []
for i in range(len(dataset)):
    new_inputs = my_model.generate(torch.tensor([dataset[i]['input_ids']]).to(my_model.device))
    words = tokenizer.batch_decode(new_inputs, skip_special_tokens=True)
    input_texts.append(dataset[i]['toxic'])
    input_texts.append(words[0])
    # result = judge.judge(words)
    # if result==0:
    #     count += 1


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
old = input_texts[0::2]
new = input_texts[1::2]

In [ ]:
import csv

# with open('test_result_two/model_with_dpo.csv', 'w', newline='') as file:
with open('model_with_dpo_onlypass.csv', 'w', newline='') as file:
# with open('test_result_two/based_model.csv', 'w', newline='') as file:
    
    writer = csv.writer(file)
    writer.writerow(['old', 'new'])
    # Optional: If you want each string in its own row
    for i in range(len(old)):  # Loop through the length of the first column

        writer.writerow([old[i], new[i]])


In [1]:

import numpy as np
from datasets import Dataset, load_dataset
import datasets
import pandas as pd
from datasets import Dataset
import random
import numpy as np
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer
import torch
from transformers import BitsAndBytesConfig
from torch import nn
from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Tuple, Union
import torch.nn.functional as F


/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [27]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

# load tokenizer and model weights
tokenizer = RobertaTokenizer.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')
model = RobertaForSequenceClassification.from_pretrained('SkolkovoInstitute/roberta_toxicity_classifier')

# prepare the input
# batch = tokenizer.encode('you are amazing', return_tensors='pt')



Some weights of the model checkpoint at SkolkovoInstitute/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
dataset = load_dataset('csv', data_files='test_result_two/model_with_dpo.csv', split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [33]:
print(dataset)

Dataset({
    features: ['old', 'new'],
    num_rows: 500
})


In [34]:
sum = 0

for i in range(len(dataset['new'])):
    batch = tokenizer.encode(dataset['new'][i], return_tensors='pt')
    result = model(batch)
    if torch.softmax(result.logits[0], dim=0)[0] > 0.8:
        sum += 1


In [35]:
print(sum/500)

0.456


In [14]:
batch = tokenizer.encode(dataset['old'][0], return_tensors='pt')

In [15]:
result = model(batch)

In [16]:
print(result)

SequenceClassifierOutput(loss=None, logits=tensor([[-2.3960,  2.3409]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [11]:
print(torch.softmax(result.logits[0], dim=0))

tensor([0.5105, 0.4895], grad_fn=<SoftmaxBackward0>)


In [57]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

# Example texts
# reference = [['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']]
# candidate = ['the', 'fast', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

dataset = load_dataset('csv', data_files='test_result_two/model_with_data.csv', split='train')

# Calculate BLEU score
# score = sentence_bleu(reference, candidate)
# print("BLEU score:", score)


In [58]:
reference = [['thequick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']]
candidate = ['the', 'fast', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']

# Calculate BLEU score
score = sentence_bleu(reference, candidate)
print("BLEU score:", score)

BLEU score: 0.7259795291154771


In [59]:
reference = dataset['old']
candidate = dataset['new']

In [60]:
sum = 0

for i in range(len(dataset)):
    sum += sentence_bleu([reference[i]], candidate[i])
    
print(sum/5)

69.43348976166702
